In [1]:
from utilities import *
from plot_utilities import *

1. given best guess x0
2. do a 1d search along r with decimal point precision 1 within general area where eta was nice
3. do precision 1 search along eta 
4. precision 2 search along r
5. 


In [14]:
def compute_prior_cdf(r, eta, n_samples = 10000, tail_bound = 0.05, tail_percent = 0.01, scale = 1, scipy_int=True, support = False):

    '''
    Returns PPoly-type function that approximates the prior CDF of the signal x
    r : shape parameter controlling rate of exponentional decay
    eta : controls roundedness of peak, and hence sparsity
    scale : scale parameter
    n_samples : number of points used to numerically approximate CDF
    tail_bound : Uses Chebyshev's Inequality to bound the region of the CDF that is outside the coverage of xs
    n_tail : Sets the number of points tha lie outside the coverage of xs to approximate tails if need be

    Usage:
    new_cdf = compute_prior_cdf(r = 0.1, eta = 0.001)
    new_cdf(0.5343) returns CDF
    Can also accept arrays
    '''
    
    beta = (eta + 1.5)/r 
    var_prior = scale * scipy.special.gamma((eta + 1.5 + 2)/r)/scipy.special.gamma(beta)
    cheby = np.sqrt(np.round(var_prior/(tail_bound)))
    

    n_tail = int(n_samples*tail_percent)
    
    x_max = min(99, cheby) # introduced additional bound in case chebyshev is unwieldy
    if cheby < 120:
        n_tail = 0
        print("No Tail")
    

    xs = np.linspace(-x_max, x_max, n_samples-2*n_tail)
    xs = np.append(-np.logspace(np.log10(cheby), 2, n_tail), xs)
    xs = np.append(xs, np.logspace(2, np.log10(cheby), n_tail))
    prior_pdf = np.full(xs.shape, np.nan)

    for j, x in enumerate(xs):

        def gauss_density(theta):
            return (1./(np.sqrt(2*np.pi)*theta)) * np.exp(-0.5*(x/theta)**2)

        def gen_gamma_density(theta):
            return (r/scipy.special.gamma(beta)) * (1/scale) * (theta/scale)**(r*beta - 1) * np.exp(-(theta/scale)**r)

        def integrand(theta):
            return gauss_density(theta) * gen_gamma_density(theta)

        if scipy_int:
            prior_pdf[j] = integrate.quad(integrand, 0, np.inf)[0]
        else:
            prior_pdf[j] = eng.compute_prior(float(r), float(eta), float(x), nargout=1)

    prior_cdf = np.zeros_like(prior_pdf)
    prior_cdf[0] = 0
    for i in range(1, len(xs)):
        prior_cdf[i] = (interpolate.CubicSpline(x = xs[:i+1], y = prior_pdf[:i+1])).integrate(xs[0], xs[i])+0

        # Alternative with Simpson's: prior_cdf[i] = integrate.simps(prior_pdf[:i+1], xs[:i+1])
    normalizer = prior_cdf[-1]
    first = prior_cdf[1]
    assert 1.05 > normalizer > 0.95
    assert 0.05 > first > -0.05
    prior_cdf = prior_cdf/normalizer   

    k = int(0.01*n_samples)
    zero_padding = np.zeros(k)
    ones_padding = np.ones(k)

    pad_max = max(10e5, np.round(cheby ** 2))


    prior_cdf = np.append(zero_padding, prior_cdf)
    xs_pad = np.append(np.linspace(-pad_max, xs[0] - 1e-5, k), xs)

    prior_cdf = np.append(prior_cdf, ones_padding)
    xs_pad = np.append(xs_pad, np.linspace(xs[-1] + 1e-5, pad_max, k))

    poly = interpolate.CubicSpline(x = xs_pad, y = prior_cdf)
    
    if support:
        return xs, poly
    else:
        return poly

In [34]:
def kstest_brute(params, *other):
    r, eta = params
    x, layer, return_loc= other
    cdf = compute_prior_cdf(r, eta, 10000)
    print(r, eta)
    with open(os.path.join(os.getcwd(),f'CDFs\\layer{layer}_10000\\{r}_{eta}.pickle'), 'wb') as handle:
        pickle.dump(cdf, handle)
    return kstest_custom(x, cdf, return_loc)


In [2]:
def line_search(obs_x, x0, layer, along_r = True, precision = 1):
    cdfs = dict()
    
    layer_cdfs = combine_pickles(f'layer{layer}_10000')
    
    if along_r:
        eta = x0[1]
        min_r = x0[0]
        min_dist, max_pval = kstest_custom(obs_x, compute_prior_cdf(min_r, eta, 10000))
        unit = 10**(-precision)

        r_range = np.arange(max(min_r - 10*unit, 0), min_r + 11*unit, unit)
        for r in r_range:
            r = round_to_sigfigs(r, precision + 3)
            if (r, eta) in layer_cdfs:
                cdf = layer_cdfs[(r, eta)]
            else:
                cdf = compute_prior_cdf(r, eta, 10000)
                with open(os.path.join(os.getcwd(),f'CDFs\\layer{layer}_10000\\{r}_{eta}-{eta}.pickle'), 'wb') as handle:
                    pickle.dump(cdf, handle)
            dist, pval = kstest_custom(obs_x, cdf)
            if dist < min_dist:
                min_dist = min(min_dist, dist)
                min_r = r
                max_pval = pval
                print(r)
        return (min_r, eta), min_dist, max_pval
    else:
        r = x0[0]
        min_eta = x0[1]
        min_dist, max_pval = kstest_custom(obs_x, compute_prior_cdf(r, min_eta, 10000))
        unit = 10**(-precision)
        eta_start = round_to_sigfigs(max(min_eta - 10*unit, 0), precision + 3)
        eta_end = round_to_sigfigs(max(min_eta + 11*unit, 0), precision + 3)
        eta_range = np.arange(eta_start, eta_end, unit)
        eta_cdfs = dict()
        for eta in eta_range:
            eta = round_to_sigfigs(eta, precision + 3)
            if (r, eta) in layer_cdfs:
                cdf = layer_cdfs[(r, eta)]
            else:
                cdf = compute_prior_cdf(r, eta, 10000)
                
            dist, pval = kstest_custom(obs_x, cdf)
            if dist < min_dist:
                min_dist = min(min_dist, dist)
                min_eta = eta
                max_pval = pval
                print(eta)
            with open(os.path.join(os.getcwd(),f'CDFs\\layer{layer}_10000\\{r}_{eta_start}-{eta_end}'), 'wb') as handle:
                pickle.dump(eta_cdfs, handle)
                
        return (r, min_eta), min_dist, max_pval

In [6]:
obs_x_dict

{2: array([-1.63935755, -8.7064424 , -7.57802165, ..., 63.2686166 ,
        -0.29168582, 11.5387479 ]),
 3: array([-5.13484135, -6.0903256 , 20.42968093, ..., -8.61188087,
         3.52167733,  2.64876462]),
 4: array([-3.91081923, -2.58042207, -4.2417638 , ..., -5.34614355,
        -2.38210087, 20.68395628]),
 5: array([-10.92873517,   1.11953819,   0.13805116, ...,  -3.32864995,
          3.50594918,   5.956682  ]),
 6: array([-0.78670251, -2.77615214, -0.59759133, ...,  1.50418374,
         0.10866727,  1.42983245]),
 7: array([ 0.82452475, -0.9606848 , -0.10590226, ...,  0.99516339,
         1.15530462,  1.07523401]),
 8: array([ 0.37822236,  0.12103116, -0.46899573, ...,  0.18301855,
         0.25165051,  0.38891443])}

In [5]:
obs_x_dict = pd.read_pickle('panoptic\\obs_x_dict.pickle')
line_search(obs_x_dict[2], [0.601, 3.16], 2, precision = 5)

TypeError: unsupported operand type(s) for |: 'dict' and 'CubicSpline'

In [54]:
line_search(obs_x_dict[2], [0.601, 3.16], 2, precision = 5, along_r = False)

-5.232436036682703e-44 1.0
-5.232573095986627e-44 1.0
-5.232630928198872e-44 1.0
-5.232688760889354e-44 1.0
-5.232746594058049e-44 1.0
-5.232804427704925e-44 1.0
-5.232146895589987e-44 1.0
-5.232204722852148e-44 1.0
-5.23226255059255e-44 1.0
-5.232320378811095e-44 1.0
-5.232378207507842e-44 1.0
3.16
-5.231778579714465e-44 1.0
-5.231836402505374e-44 1.0
-5.231894225774576e-44 1.0
-5.231952049521846e-44 1.0
-5.232009873747225e-44 1.0
-5.232067690656568e-44 1.0
-5.231410308822711e-44 1.0
-5.231468120011727e-44 1.0
-5.231525938812957e-44 1.0
-5.231583758092222e-44 1.0
-5.231641577849569e-44 1.0


((0.601, 3.16), 0.0370990182590879, 0.0003074596006262201)

In [ ]:
def coord_descent(x0)

In [ ]:
def finer_optimize(x0, axis = 'r', precision = 2):

    all_r = np.arange(0.59, 0.61, 0.001)
    all_eta = np.arange(3.14, 3.18, 0.01)
    num_points = 10000
    add_cdfs(r_range = all_r, eta_range = all_eta, n_samples = num_points, scipy_int=True, folder_name='layer2_')

In [24]:
all_cdfs = combine_pickles('100')

with open(f'panoptic\\obs_x_dict.pickle', 'rb') as handle:
    obs_x_dict = pickle.load(handle)
with open(f'panoptic\\df_dict_cdfs_100000_0.1-2.9-0.1_0-4-0.2.pickle', 'rb') as handle:
    df_dict = pickle.load(handle)


In [85]:
old_best = pd.read_csv(os.path.join(os.getcwd(), 'panoptic\\CSVs\\best_params_df_cdfs_100000_0.1-2.9-0.1_0-4-0.2.csv')).set_index('layer')
old_best

,num_samples,r,eta,kstest_stat,kstest_pval
layer,,,,,
2.0,3180.0,0.6,3.2,0.037981,2.016182e-04
3.0,12720.0,0.7,3.6,0.021390,1.736009e-05
4.0,50880.0,0.8,3.0,0.011778,1.468903e-06
5.0,203520.0,0.9,1.6,0.003276,2.527426e-02
6.0,814080.0,1.0,0.2,0.008452,6.021976e-51
7.0,3256320.0,4.6,0.0,0.024893,0.000000e+00
8.0,13025280.0,5.8,0.0,0.171258,0.000000e+00


In [7]:
best_params_df = pd.read_csv(os.path.join(os.getcwd(), 'panoptic\\CSVs\\best_params_df_finer_grid.csv')).set_index(['layer']).sort_index()
best_params_df

,num_samples,r,eta,kstest_stat,kstest_pval
layer,,,,,
2.0,3180.0,0.601,3.18,0.036228,4.619069e-04
3.0,12720.0,0.705,3.63,0.018491,3.295114e-04
4.0,50880.0,0.790,0.24,0.218044,0.000000e+00
5.0,203520.0,0.904,1.62,0.003040,4.637489e-02
6.0,814080.0,0.990,0.24,0.005327,1.720600e-20


In [ ]:
all_cdfs = combine_pickles(f'layer{layer}_10000')

In [ ]:
layer = 2
layer_cdfs = combine_pickles(f'layer{layer}_10000')
layer_df = pd.DataFrame({'(r,eta),cdf' : all_cdfs.items()})
layer_df['r'] = pd.Series(layer_df["(r,eta),cdf"].str[0].str[0])
layer_df['eta'] = pd.Series(layer_df["(r,eta),cdf"].str[0].str[1])
layer_df['cdf'] = pd.Series(layer_df["(r,eta),cdf"].str[1])



In [17]:
from scipy import optimize

def minimize_kstest_stat(layer = None, best_params_df = None, x = [0], x0 = [0, 0]):
    if layer:
        x = np.sort(obs_x_dict[layer])
        x0 = [best_params_df.loc[layer]['r'], best_params_df.loc[layer]['eta']]
    print(x, x0)
    n = len(x)
    history = []

    def kstest_stat(params):
        r = params[0]
        eta = params[1]
        cdf = compute_prior_cdf(r, eta, 10000)
        history.append((r, eta))
        print(r, eta)
        cdfvals = cdf(x)
        dplus, dminus = np.max(np.arange(1.0, n + 1) / n - cdfvals), np.max(cdfvals - np.arange(0.0, n)/n)
        return max(dplus, dminus)
    
    print(x0, bounds=[(x0[0]-0.1, x0[0]+0.1), (x[1]-0.2, x[1]+0.2)])
    optimized = optimize.minimize(kstest_stat, x0, bounds=[(x0[0]-0.1, x0[0]+0.1), (x[1]-0.2, x[1]+0.2)], tol=1e-3)
    x_prime = optimized['x']
    msg = optimized['message']
    return x_prime, msg, history

In [18]:
params, msg, hist = minimize_kstest_stat(layer = 3, best_params_df=best_params_df)
params, msg

[-76.13432886 -63.02074639 -60.52416619 ...  65.19608846  65.26965934
  69.44498554] [0.7, 3.6]


ValueError: `x` must be strictly increasing sequence.

In [18]:


for layer in np.arange(2,9):
    
    fixed_x = np.sort(obs_x_dict[layer])
    n = len(fixed_x)
    history_dict = dict()

    def kstest_stat(params):
        r = params[0]
        eta = params[1]
        cdf = compute_prior_cdf(r, eta, 10000)
        history_dict[layer].append((r, eta))
        print(r, eta)
        cdfvals = cdf(fixed_x)
        dplus, dminus = np.max(np.arange(1.0, n + 1) / n - cdfvals), np.max(cdfvals - np.arange(0.0, n)/n)
        return max(dplus, dminus)

optimize.minimize(kstest_stat, [0.7, 3.6], bounds=[(0.5, 0.9), (3.5, 3.6)], tol=1e-5)
    


In [39]:
from scipy import optimize
optimize.minimize(kstest_stat, [0.7, 3.6], bounds=[(0.5, 0.9), (3.4, 3.8)], tol=1)['x']

0.7 3.6
0.70000001 3.6
0.7 3.60000001


array([0.7, 3.6])